## 概要
季節ラベルのない俳句に対して、その俳句の季節を予測する。

## 方法
1. 俳句をベクトル化することで、各俳句の類似度を数値化できる。
2. 類似している俳句からクラスラベル（季節）を推測する。

## 注意点
word2vecでベクトル化するのは、word即ち単語であって、文章ではない

## 参考資料
- [初めての自然言語処理 類似文書検索の手法と精度比較](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part1.html)
- [understanding word vectors](https://gist.github.com/aparrish/2f562e3737544cf29aaf1af30362f469)

In [1]:
import numpy as np
import pandas as pd
import MeCab
import pickle
import sys, os 
sys.path.append(os.path.join(os.path.curdir, '../../'))
from common.utils import *
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
haijins = [
    'bashou',
    'buson',
    'issa',
    'siki',
    'souseki',
    'ryunosuke',
    'kyoshi',
    'housai',
    'hekigotoh',
    'hisajo',
]

df_list = []
for haijin in haijins:
    df = pd.read_csv('../../data/Basyo/{}.csv'.format(haijin), index_col=0)
    df['作者'] = haijin
    df_list.append(df)
    
haiku_df = pd.concat(df_list, axis=0, ignore_index=True)
print(haiku_df.shape)
haiku_df.head()

(14495, 4)


,俳句,季語,季節,作者
0,姥桜さくや老後の思ひ出,NaN,NaN,bashou
1,年は人にとらせていつも若夷,NaN,NaN,bashou
2,花の顔に晴うてしてや朧月,花,春,bashou
3,盛なる梅にす手引風も哉,梅,春,bashou
4,あち東風や面々さばき柳髪,東風,春,bashou


In [ ]:
m = MeCab.Tagger('-Ochasen')

wakachi = lambda x: wakachi_gaki(m, x)
haiku_df['分かち書き'] = haiku_df['俳句'].apply(wakachi)

noun_extractor = lambda x: ' '.join(text2nlist(x))
haiku_df['名詞'] = haiku_df['俳句'].apply(noun_extractor)

# with open('../../data/pickles/haiku_df.pickle', 'wb') as f:
#     pickle.dump(haiku_df, f)

In [ ]:
with open('../../data/pickles/haiku_df.pickle', 'rb') as f:
    haiku_df = pickle.load(f)
haiku_df.head()

### bag of words

In [9]:
cv = CountVectorizer()
bow = cv.fit_transform(haiku_df['名詞'].tolist())
print(bow.toarray().shape)
bow.toarray()

(14495, 7061)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### TF-IDF

In [13]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(haiku_df['名詞'].tolist())
print(X.toarray().shape)
X.toarray()[0]

(14495, 7061)


array([0., 0., 0., ..., 0., 0., 0.])

### word2vec

In [8]:
sentences = []
for wakachi in haiku_df['分かち書き']:
    w_list = wakachi.split(' ')
    sentences.append(w_list)

sentences

[['姥桜', 'さく', 'や', '老後', 'の', '思ひ', '出'],
 ['年', 'は', '人', 'に', 'とら', 'せ', 'て', 'いつも', '若', '夷'],
 ['花', 'の', '顔', 'に', '晴', 'うて', 'し', 'て', 'や', '朧月'],
 ['盛', 'なる', '梅', 'に', 'す', '手引', '風', 'も', '哉'],
 ['あ', 'ち', '東風', 'や', '面々', 'さばき', '柳', '髪'],
 ['餅', '雪', 'を', 'しら', '糸', 'と', 'なす', '柳', '哉'],
 ['花', 'に', 'あか', 'ぬ', '嘆', 'や', 'こ', 'ちの', 'う', 'たぶ', 'くろ'],
 ['なつ', 'ちかし', '其口', 'た', 'ば', 'へ', '花', 'の', '風'],
 ['うかれ', 'ける', '人', 'や', '初瀬', 'の', '山桜'],
 ['糸桜', 'こ', 'や', 'か', 'へる', 'さ', 'の', '足', 'もつれ'],
 ['風', '吹け', 'ば', '尾', 'ぼ', 'そう', 'なる', 'や', '犬', '櫻'],
 ['春立', 'と', 'わら', 'は', 'も', '知', 'や', 'かざり', '縄'],
 ['き', 'て', 'も', 'みよ', '甚', 'べ', 'が', '羽織', '花', 'ごろ', 'も'],
 ['花', 'に', 'いや', 'よ', '世間', '口', 'より', '風', 'の', 'くち'],
 ['植', 'る', '事', '子', 'の', 'ごとく', 'せよ', '児', '櫻'],
 ['目', 'の', '星', 'や', '花', 'を', 'ねが', 'ひ', 'の', '糸', '櫻'],
 ['天びん', 'や', '京', '江戸', 'かけ', 'て', '千代', 'の', '春'],
 ['此梅', 'に', '牛', 'も', '初音', 'と', '鳴', 'つ', 'べし'],
 ['我', 'も', '神', 'の', 'ひさ', 'う', 'やあ', 'ふぐ', '梅', '

In [9]:
model = Word2Vec(sentences, sg=1, window=5, min_count=1)

In [14]:
for i in model.most_similar('かなしみ'):
    print(i)

('一盞', 0.9924553632736206)
('じみ', 0.9924271106719971)
('ゆらぐ', 0.9923542737960815)
('徐', 0.992311954498291)
('会釈', 0.992301881313324)
('講', 0.9922609925270081)
('まれ', 0.9921238422393799)
('いざ', 0.992029070854187)
('褄', 0.991996705532074)
('せめて', 0.9919725656509399)


c:\users\junni\venv\nlp_env\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [15]:
sent1 = haiku_df.loc[0, ['分かち書き']].tolist()[0]
sent2 = haiku_df.loc[1, ['分かち書き']].tolist()[0]
distance = model.wmdistance(sent1, sent2)
print(sent1)
print(sent2)
print(distance)

姥桜 さく や 老後 の 思ひ 出
年 は 人 に とら せ て いつも 若 夷
0.6955133489913699


c:\users\junni\venv\nlp_env\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


### 俳句の季節推測

In [16]:
haiku_df.head()

,俳句,季語,季節,作者,分かち書き,名詞
0,姥桜さくや老後の思ひ出,NaN,NaN,bashou,姥桜 さく や 老後 の 思ひ 出,姥桜 さく 老後 出
1,年は人にとらせていつも若夷,NaN,NaN,bashou,年 は 人 に とら せ て いつも 若 夷,年 人 夷
2,花の顔に晴うてしてや朧月,花,春,bashou,花 の 顔 に 晴 うて し て や 朧月,花 顔 晴 朧月
3,盛なる梅にす手引風も哉,梅,春,bashou,盛 なる 梅 に す 手引 風 も 哉,盛 梅 手引 風 哉
4,あち東風や面々さばき柳髪,東風,春,bashou,あ ち 東風 や 面々 さばき 柳 髪,東風 面々 さばき 柳 髪


In [23]:
print('季節ラベルあり俳句数: ', haiku_df[haiku_df['季節'].isnull() == False].shape[0])
print('季節ラベルなし俳句数: ', haiku_df[haiku_df['季節'].isnull()].shape[0])

季節ラベルあり俳句数:  11288
季節ラベルなし俳句数:  3207


In [3]:
season_df = haiku_df[haiku_df['季節'].isnull() == False]
season_df = season_df.reset_index(drop=True)
non_season_df = haiku_df[haiku_df['季節'].isnull()]
non_season_df = non_season_df.reset_index(drop=True)

# pickle化
season_df.to_pickle('../../data/pickles/season.pickle')
non_season_df.to_pickle('../../data/pickles/non_season.pickle')

In [48]:
from collections import Counter
def haiku_similarity(model, h1, h2):
    distance = model.wmdistance(h1, h2)
    return distance

def season_estimation(model, haiku):
    s_haiku_list = season_df['分かち書き'].tolist()
    estimation_df = season_df.copy()
    estimation_df['類似度'] = estimation_df['分かち書き'].apply(lambda x: haiku_similarity(model, haiku, x,))
    estimation_df = estimation_df.sort_values('類似度', ascending=True)
    counter = Counter(estimation_df.loc[:4, '季節'].tolist())
    season = counter.most_common()[0][0]
    
    return season

In [49]:
haiku = non_season_df.loc[:0]['分かち書き'].tolist()[0]
haiku
season_estimation(model, haiku)

c:\users\junni\venv\nlp_env\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


類似俳句：
['川縁や炬燵の酔をさます人', '凩や白樺の魔火さそふ森', '藤さくや木辻の君が夕粧ひ', '梨咲くやいくさのあとの崩れ家', 'おもひ立木曾や四月のさくら狩', '鍬さげて神農顔やきくの花', '梅さくや平親王の御月夜', '蓮の花さくや淋しき停車場', '嚊殿に盃さすや菊の酒', '葉鶏頭に土の固さや水沁まず', '裏戸近く夕汐さすや茨の花', 'うの花や蛙葬る明り先', '朝寒やさざ波白き川の上', '木槿さく畑の径や木幡山', '日の道や葵傾くさ月あめ', '麦の出来悪しと鳴くや行々子', '畑打音やあらしのさくら麻', 'さみだれや蚕煩ふ桑の畑', '貧しさや葉生姜多き夜の市', '夕立や並んでさわぐ馬の尻', '閑さや岩にしみ入る蝉の声', '炎天や蟻這ひ上る人の足', '赤根さす昼や若葉の木の曲り', '奈良の鹿やせてことさら神々し', '花さくや欲のうき世の片隅に', '我も神のひさうやあふぐ梅の花', '街道の貫く町や粟を干す', '奈良の町の昔くさしや朧月', '春寒や刻み鋭き小菊の芽', '大雪や印の竿を鳴く烏', '夕顔に平壌のいくさ物語れ', '寒菊や年々同じ庭の隅', '白菊の花でこさばや濁り酒', '芭蕉忌や茶の花折つて奉る', '暁や霧わけ出る鹿の角', '稲妻の崩れたあとや夕嵐', '帰り咲く八重の桜や法隆寺', 'どくだみの花の暑さや総後架', '咲乱す桃の中より初桜', 'すさましや花見戻りの下駄の音', '粥すする匙の重さやちちろ虫', 'やうやうに残る暑さも萩の露', '桃さくや三寸程の上り鮎', '長閑さや浅間のけぶり昼の月', '飯櫃の蛍追ひ出す夜舟哉', '見てさへや惣身にひびく寒の水', '探梅や序でに僧に届けもの', '鶯や懐の子も口を明く', '勢ひやひしめく江戸の年の市', '蚊遣火やこの時出づる蚊喰鳥', '赤光や砲車ひきゆく馬の汗', '蓬莱の松にさしけり初日の出', '青ざしや草餅の穂に出つらん', '幕垂れて玉座くらさや雨の雛', '雪よけの長き廂や蚊喰鳥', '冬枯や巡査に吠ゆる里の犬', 'はやくさけ九日も近し菊の花', '夜桜や大雪洞の空うつり', '菜の花や化されてゐる女の子', '咲き絶えし薔薇の心や冬籠', '笋や柑子ををしむ垣の外', '宵越のとうふ明りや蚊のさはぐ', '

'秋'